# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [41]:
#com o selenium tudo roda em segundo (ou terceiro, quarto, quinto...) plano, então pode rodar enquanto você usa outras coisas
#o selenium não precisa esperar a página carregar, ele vai fazendo enquanto não da nenhum erro
!pip install selenium

#webdriver 
#chrome -> chromedriver (instalei e coloquei na pasta do jupyter)

In [42]:
#Passo a passo
#Passo 1 - abrir o navegador
#O selenium só consegue fazer as coisas no navegador
from selenium import webdriver
navegador = webdriver.Chrome()

#get para entrar em um site
navegador.get('https://www.google.com.br/')



In [43]:
#Passo 2 - Importar base de dados
import pandas as pd #pra ler a base de dados que está no excel

tabela = pd.read_excel('commodities.xlsx') #não precisa mostrar o caminho inteiro até o arquivo pq ta na mesma pasta do projeto3
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Trigo,1549.11,NaN,NaN
9,Tilápia,9.05,NaN,NaN


In [44]:
#Passo 3 - Para cada produto da base de dados:

#Passo 4 - Pesquisar o preço do produto

#primeiro fazer para um item e depois colocar para todos - começando com milho
#link = 'https://www.melhorcambio.com/milho-hoje'
#navegador.get(link)

#preco_milho = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
#print(preco_milho)

#automatizar pra tabela toda
for linha in tabela.index: #index - ele busca todas as linhas 
    produto = tabela.loc[linha, 'Produto']
    print(produto)
    
#o que muda é o que ta no link  -> Produto-hoje -> f na frente

#erro pq alguns tem acento -> editar o nome do produto pra tirar caracteres especiais
#dá pra mudar tudo em um código só, 
#import unicodedata
#produto=unicodedata.normalize ('NFKD', produto).encode('ascii', 'ignore')


#mas dessa vez fizemos um por um:

    produto = produto.replace('ó', 'o').replace('ã','a').replace('á','a').replace('ç','c').replace('ú','u').replace('é','e')
    
    link = f'https://www.melhorcambio.com/{produto}-hoje'
    navegador.get(link)

    preco = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
    
#editar o preço pq uns são '.' e outros ','
    preco = preco.replace('.','').replace(',', '.')
    
    print(preco)

#garantir que estejam sendo lidos como números
    tabela.loc[linha,'Preço Atual'] = float(preco)
    
#Passo 5 - Atualizar o preço na base de dados
    
    
display(tabela)

#.click() -> clicar
#.send_keys('texto') -> escrever
#.get_attribute() -> pegar um valor

Milho
85.49
Soja
162.64
Boi
276.05
Petróleo
391.28
Algodão
498.88
Açúcar
134.64
Café
1087.87
Ouro
324.43
Trigo
1546.08
Tilápia
9.05


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,85.49,NaN
1,Soja,163.59,162.64,NaN
2,Boi,282.20,276.05,NaN
3,Petróleo,424.37,391.28,NaN
4,Algodão,497.76,498.88,NaN
5,Açúcar,136.23,134.64,NaN
6,Café,1092.87,1087.87,NaN
7,Ouro,321.77,324.43,NaN
8,Trigo,1549.11,1546.08,NaN
9,Tilápia,9.05,9.05,NaN


In [45]:
#Passo 6 - Decidir quais produtos comprar
#preencher a coluna de comprar
tabela['Comprar'] = tabela['Preço Atual'] <= tabela['Preço Ideal']
display(tabela)

#exportar a base para o excel
#tabela.to_excel('commodities.xlsx') - substitui o arquivo original

tabela.to_excel('commodities_atualizado.xlsx', index=False)

#index=Falso é para deletar a tabela de índice

#fechar navegador
navegador.quit()



,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,85.49,False
1,Soja,163.59,162.64,True
2,Boi,282.20,276.05,True
3,Petróleo,424.37,391.28,True
4,Algodão,497.76,498.88,False
5,Açúcar,136.23,134.64,True
6,Café,1092.87,1087.87,True
7,Ouro,321.77,324.43,False
8,Trigo,1549.11,1546.08,True
9,Tilápia,9.05,9.05,True
